In [1]:
import yfinance as yf
import pandas as pd
import numpy as np

### Utility Functions

In [2]:
def get_volatility(symbol, period='45d'):
    
    # Get NSE Historic Data
    ticker = yf.Ticker(symbol)
    df = ticker.history(period=period)['Close'].reset_index()

    # Calculate Daily Returns
    df['Prev Close'] = df['Close'].shift(1)
    df['Daily Rt'] = np.log(df['Close'] / df['Prev Close'])

    # Calculate Daily and Annualized Volatility
    daily_volatility = df['Daily Rt'].std()
    annual_volatility = daily_volatility * np.sqrt(252)

    return daily_volatility, annual_volatility

### Experiment

In [4]:
# URL to get the list of NSE stocks
nse_tickers_url = '../data/static/ind_nifty50list.csv'

# Read the data from the URL
tickers_df = pd.read_csv(nse_tickers_url)

# Extract the Symbol column
tickers = tickers_df['Symbol'].tolist()

# Print all NSE tickers
vol_df = []
for ticker in tickers:
   dv, av = get_volatility(ticker+'.NS')
   tmp = {
      'Symbol': ticker,
      'Daily Volatility': dv,
      'Annual Volatility': av
   }
   vol_df.append(tmp)

vol_df = pd.DataFrame(vol_df)

In [5]:
vol_df

,Symbol,Daily Volatility,Annual Volatility
0,ADANIENT,0.021944,0.348346
1,ADANIPORTS,0.019141,0.303857
2,APOLLOHOSP,0.016252,0.257998
3,ASIANPAINT,0.014177,0.225048
4,AXISBANK,0.014272,0.226566
5,BAJAJ-AUTO,0.014903,0.236585
6,BAJFINANCE,0.017784,0.282305
7,BAJAJFINSV,0.012840,0.203825
8,BPCL,0.022513,0.357383
9,BHARTIARTL,0.014560,0.231133


### Rough

In [79]:
get_volatility('ZYDUSLIFE.NS')

(0.020992579894727276, 0.33324687467461717)

In [8]:
ticker = yf.Ticker('ZYDUSLIFE.NS')

In [69]:
df = ticker.history(period='45d')['Close'].reset_index()

In [70]:
df['Prev Close'] = df['Close'].shift(1)

In [71]:
df['Daily Rt'] = np.log(df['Close'] / df['Prev Close'])

In [72]:
df.to_csv('../data/test/zyduslife.csv', index=False)

In [73]:
daily_volatility = df['Daily Rt'].std()
daily_volatility

0.020992579894727276

In [74]:
annual_volatility = daily_volatility * np.sqrt(252)
annual_volatility

0.33324687467461717

In [75]:
df.shape

(46, 4)